# Prep environment

In [ ]:
#Import relevant packages
import numpy as np
import pandas as pd
from matplotlib import rcParams
import os
import scanpy as sc

import matplotlib as mpl
import matplotlib.pyplot as plt

#For nice color schemes
import cmocean

#For barplots
import seaborn as sns

In [ ]:
#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40

In [ ]:
# for optimal transport
import scipy
from scipy.spatial.distance import cdist

from sklearn.cluster import AgglomerativeClustering, SpectralClustering

from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.metrics import normalized_mutual_info_score

import sklearn

from tqdm import tqdm

import otscomics

In [ ]:
#Set fontsize
plt.rcParams.update({'font.size': 20})

In [ ]:
#Set wd 
os.chdir('/hpc/group/goldsteinlab/Python')

In [ ]:
#Show specific size of pandas dataframe when produced
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [1]:
%matplotlib inline

# Read in datasets

In [ ]:
# For full OE + primary RPM and RPMA tumors
adata=sc.read_h5ad('OE_atlas_with_ONB_tumors_all_cells.h5ad')

In [ ]:
# For just RPM and RPMA primary tumor cells
adata=sc.read_h5ad('Primary_ONB_tumors_only_scvi.h5ad')

# Dotplots

In [ ]:
# plot dotplot showing expression of canonical markers across cell types

genes=['Krt5', 'Krt17', 'Sox2', 'Trp63', #HBC
      'Hes6', 'Kit', 'Ascl1', 'Mki67', 'Top2a', #GBC
      'Neurod1', 'Neurog1', 'Sox11', 'Zfp423', 'Ebf1', 'Lhx2', #INP
      'Olig2', 'Gng8', 'Ebf4', 'Tubb3', #iOSN
      'Omp', 'Gng13', 'Rtp1', 'Gfy', 'Stoml3', #mOSN
      'S100b', 'Plp1', 'Apoe', #OEC
      'Pou2f3', 'Sox9', 'Trpm5', 'Chat', 'Avil', 'Krt8', 'Krt18', #MV tuft like
      'Foxi1', 'Cftr', 'Ascl3', 'Smbd1', 'Stap1', 'Moxd1', 'Atp6v0d2', 'Krt8', 'Krt18', #MV ionocyte like
       'Sox9', 'Sox10', #BG
      'Cyp2j6', 'Cxcl17', 'Ermn', 'Sox2', 'Krt8', 'Krt18', #Sus
      'Dcn', 'Pdgfra', 'Vim', #Fibroblasts
       'Sox17', 'Vwf', 'Tagln', 'Eng', #pericytes
        'Ptprc', 'Cd68', 'C1qa', 'C1qb', 'Cd14', 'Adgre1', #myeloid
       'Ptprc', 'Cd3d', 'Cd3e', 'Cd4', 'Cd8a', #lymphoid
      'Myc', 'Cas9', 'fLuc' #tumor markers
      ]

fig, ax = plt.subplots(figsize=(7,22))
sc.pl.dotplot(adata, genes, groupby='cluster_names', layer='norm', ax=ax, cmap='Purples',
             swap_axes=True,
             save=False, categories_order=['HBC', 'GBC', 'INP', 
                                          'iOSN', 'mOSN', 'Olfactory ensheathing', 'MV tuft-like', 'MV ionocyte-like',
                                         'Bowmans Glands', 'Sustentacular', 'Fibroblast', 'Pericyte',
                                          'Myeloid', 'Lymphoid', 'RPM tumor', 'RPMA tumor'],
              standard_scale='var',
              mean_only_expressed=False,
             vcenter=0.5)

In [2]:
# dotplot specific for RPM and RPMA analyses
# using adata object with just RPM and RPMA tumors

# for ease of visualization, normalize, log1p, and scale .X
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.scale(adata)

# can now generate dotplot, like so:
genes=['Krt5', 'Krt17', 'Sox2', 'Trp63', #HBC
      'Hes6', 'Kit', 'Ascl1', 'Mki67', 'Top2a', #GBC
      'Neurod1', 'Neurog1', 'Sox11', 'Zfp423', 'Ebf1', 'Lhx2', #INP
      'Olig2', 'Gng8', 'Ebf4', 'Tubb3', #iOSN
      'Omp', 'Gng13', 'Rtp1', 'Gfy', 'Stoml3', #mOSN
      'Pou2f3', 'Sox9', 'Trpm5', 'Avil', 'Chat', 'Krt8', 'Krt18', #MV tuft like
      'Foxi1', 'Cftr', 'Smbd1', 'Stap1', 'Moxd1', 'Atp6v0d2', 'Asgr1', 'Ascl3', 'Krt8', 'Krt18', #MV ionocyte like
       'Sox9', 'Sox10', #BG
      'Cyp2j6', 'Cxcl17', 'Ermn', 'Sox2', 'Krt8', 'Krt18', #Sus
      ]

fig, ax = plt.subplots(figsize=(13.5,3))
sc.pl.dotplot(adata, genes, groupby='tumor_type', ax=ax, cmap='Purples',
             swap_axes=False,
             save=True, categories_order=['RPM', 'RPMA'],
              standard_scale=False,
              mean_only_expressed=False,
             vcenter=0.5, vmin=-3, vmax=2)

# Cell cycle genes

In [ ]:
# Cell cycle identification

#Normalize X and then log and scale before scoring
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)
sc.pp.scale(adata)

Note, gene lists below are from Tirosh gene set (PMC5465819), used R to convert to mouse homologs to run in mice datasets

In [ ]:
#Need to load in cell cycle genes list
s_genes_csv=pd.read_csv('/hpc/group/goldsteinlab/R/Working_directory/mouse_cell_cycle_genes_s.csv')
#Select sample/condition gene list that you want
s_genes_csv=s_genes_csv[['gene']]
#Convert df or series to list
s_genes=s_genes_csv.squeeze().str.strip().to_list()

In [ ]:
#Need to load in cell cycle genes list
g2m_genes_csv=pd.read_csv('/hpc/group/goldsteinlab/R/Working_directory/mouse_cell_cycle_genes_g2m.csv')
#Select sample/condition gene list that you want
g2m_genes_csv=g2m_genes_csv[['gene']]
#Convert df or series to list
g2m_genes=g2m_genes_csv.squeeze().str.strip().to_list()

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

In [ ]:
cell_cycle_genes=s_genes+g2m_genes

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
sc.pl.umap(adata, color="phase", legend_loc='right margin',
        ax=ax, frameon=False, save=False)

# Cell type markers

In [ ]:
# generate list of differentially expressed genes for each cell type

#First filter so only cell-types we care about
bad_clust=['Fibroblast', 'Lymphoid', 'Myeloid', 'Olfactory ensheathing', 'Pericyte']

#Filter out bad clusters
to_keep=(~adata.obs['cluster_names'].isin(bad_clust))

#Copy over to new anndata object
adata_f = adata[to_keep].copy()

In [ ]:
#Calculate highly variable genes using scVI methods
df_poisson = scvi.data.poisson_gene_selection(
    adata_f, n_top_genes=3000, batch_key="mouse_ident", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata_f.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata_f.varm['df_poisson']= df_poisson

adata_query = adata_f[:, is_hvg].copy()
print(adata_query)

In [ ]:
# scvi runs on raw data, but following hvg gene selection, need to make sure everything below is run on normalized data
adata_query.X=adata_query.layers['norm'].copy()

In [ ]:
#Now find all markers
#Find cluster markers
sc.tl.rank_genes_groups(adata_query, 'cluster_names', method='wilcoxon', use_raw=False)
pd.DataFrame(adata_query.uns['rank_genes_groups']['names']).head(10)

In [ ]:
df.to_csv('Wilcoxon_marker_genes_per_cluster.csv')

In [ ]:
#Calculate PCs
sc.pp.pca(adata_query, n_comps=30)
sc.pl.pca_variance_ratio(adata_query, n_pcs=30)

In [ ]:
#Run dendrogram
sc.tl.dendrogram(adata_query, groupby='cluster_names', n_pcs=30)

In [ ]:
# log1p and scale for visualization on heatmap
sc.pp.log1p(adata_query)
sc.pp.scale(adata_query)

In [ ]:
# randomly subsample equal numbers of cells across clusters for nicer plotting visualization

# Set the random seed for reproducibility
np.random.seed(42)

# Initialize an empty list to store the sampled cell indices
sampled_indices = []

# Iterate over each cluster
for cluster_label in adata_query.obs['cluster_names'].unique():
    # Get the cell indices belonging to the current cluster
    cluster_indices = np.where(adata_query.obs['cluster_names'] == cluster_label)[0]
    
    # Randomly sample 100 cells from the cluster
    sampled_indices.extend(np.random.choice(cluster_indices, size=100, replace=False))

# Subset the AnnData object to include only the sampled cells
sampled_adata = adata_query[sampled_indices, :]

# Print the number of sampled cells per cluster
for cluster_label in sampled_adata.obs['cluster_names'].unique():
    num_cells = np.sum(sampled_adata.obs['cluster_names'] == cluster_label)
    print(f"Cluster {cluster_label}: {num_cells} cells")

In [ ]:
# Plot rank genes groups (here clustering by annotated group name)
sc.pl.rank_genes_groups_heatmap(sampled_adata, groupby='cluster_names', n_genes=30, 
                            swap_axes=True, cmap='RdBu_r', 
                                standard_scale='obs', 
                                #vmin=-1,  vmax=1, 
                                figsize=[25,6], show_gene_labels=False,
                               min_logfoldchange=1, save=True)

# Gene set (module) scores

In [ ]:
# first for human tumor specific scores 
# see "1_bulk_RNA-Seq_atlas_UMAPs.ipynb" section "generate tumor specific signatures" for how these were made

# read in mouse orthologs (human genes were converted to mouse orthologs, as described in R script 4)
Hum_sigs=pd.read_csv('Top_hSigs.csv')

In [ ]:
# Generate individual lists

NB=Hum_sigs.NB_Top_500
SCLC=Hum_sigs.SCLC_Top_500
LUAD=Hum_sigs.LUAD_Top_500
ONB=Hum_sigs.ONB_Top_500

NB=NB.squeeze().str.strip().to_list()
SCLC=SCLC.squeeze().str.strip().to_list()
LUAD=LUAD.squeeze().str.strip().to_list()
ONB=ONB.squeeze().str.strip().to_list()


In [ ]:
# set .X to the normalized layer
adata.X = adata.layers['norm']

In [ ]:
# Add human ONB score to mouse data 
sc.tl.score_genes(adata, SCLC, score_name='SCLC_Human_Score')
sc.tl.score_genes(adata, NB, score_name='NB_Human_Score')
sc.tl.score_genes(adata, LUAD, score_name='LUAD_Human_Score')
sc.tl.score_genes(adata, ONB, score_name='ONB_Human_Score')

In [ ]:
# plot scores
with plt.rc_context({'figure.figsize': (12, 10)}):
    sc.pl.umap(
    adata2,
    color=['cluster_names','ONB_Human_Score','SCLC_Human_Score','NB_Human_Score','LUAD_Human_Score'],
    use_raw=False,
    legend_loc= "on data",legend_fontweight='medium',legend_fontsize='xx-large',
    ncols=3,
        vmin='0',
    vmax='8',
    frameon=False,
    save=False,
    s=50
)

In [3]:
# for next scores, using OE + primary tumors adata object

#Filter out non-OE lineage cells for this (ie immune, fibroblasts, non OE lineage)
bad_clust=['Fibroblast', 'Lymphoid', 'Myeloid', 'Olfactory ensheathing', 'Pericyte']

#Filter out bad clusters
to_keep=(~adata.obs['cluster_names'].isin(bad_clust))

#Copy over to new anndata object
adata = adata[to_keep].copy()

In [ ]:
#Order adata.obs for graphing downstream
category_order_list=['Bowmans Glands', 'Sustentacular', 'MV ionocyte-like', 'MV tuft-like', 'HBC', 'GBC',  'RPMA tumor', 'RPM tumor', 'INP', 'iOSN', 'mOSN']
#Reorder cluster names for plotting
adata.obs['cluster_names'].cat.set_categories(category_order_list)

In [ ]:
# set .X to normalized layer for downstream analysis
adata.X = adata.layers['norm']

next set of module scores are from human sclc Chan et al., Cancer Cell 2021 (PMC8628860)

In [ ]:
#Read in mouse Pou2f3 gene list (generated in R from human genes)
Pou_up_list= pd.read_csv('/hpc/group/goldsteinlab/R/Working_directory/JChan_CCell_Pou2f3_up_mouse_versions.csv')
Pou2f3_up=Pou_up_list[['x']]

#Convert df or series to list
Pou2f3_up_targets=Pou2f3_up.squeeze().str.strip().to_list()

In [ ]:
#Read in mouse Neurod1 gene list (generated in R from human genes)
Neurod1_up_list= pd.read_csv('/hpc/group/goldsteinlab/R/Working_directory/JChan_CCell_Neurod1_up_mouse_versions.csv')
Neurod1_up=Neurod1_up_list[['x']]
#Convert df or series to list
Neurod1_up_targets=Neurod1_up.squeeze().str.strip().to_list()

In [ ]:
#Read in mouse Ascl1 gene list (generated in R from human genes)
Ascl1_up_list= pd.read_csv('/hpc/group/goldsteinlab/R/Working_directory/JChan_CCell_Ascl1_up_mouse_versions.csv')
Ascl1_up=Ascl1_up_list[['x']]
Ascl1_up_targets=Ascl1_up.squeeze().str.strip().to_list()

In [ ]:
#Add targets to anndata object
sc.tl.score_genes(adata, Pou2f3_up_targets, score_name='human_Pou2f3_enriched')
sc.tl.score_genes(adata, Neurod1_up_targets, score_name='human_Neurod1_enriched')
sc.tl.score_genes(adata, Ascl1_up_targets, score_name='human_Ascl1_enriched')

In [ ]:
# plot UMAPs

#Plot
sc.pl.umap(
    adata,
    color=['cluster_names', 'human_Ascl1_enriched'],
    use_raw=False,
    legend_loc= "on data",
    color_map="PRGn",
    ncols=3,
    vmax='p99',
    frameon=False,
    save=False,
    s=10
)

In [ ]:
# to generate bar graphs with stats

# first create df from adata
df_score = sc.get.obs_df(adata, keys=['cluster_names', 'human_Ascl1_enriched'])

# plot
fig, ax = plt.subplots(figsize=(5,5))
ax=sns.barplot(data=df_score, x='cluster_names', y='human_Ascl1_enriched', 
               order=['Bowmans Glands', 'Sustentacular', 'MV ionocyte-like', 'MV tuft-like', 'HBC', 'GBC',  'RPMA tumor', 'RPM tumor', 'INP', 'iOSN', 'mOSN'],
            capsize=0.2,
           #errorbar=('ci', 95), 
               palette=palette)
ax, test_results=add_stat_annotation(ax, data=df_score, x='cluster_names', y='human_Ascl1_enriched', box_pairs=[('RPM tumor', 'RPMA tumor')], test='Mann-Whitney', text_format='star', loc='outside', verbose=2)
plt.xticks(rotation=90)

Neurod1 ChIP target scores were generated in the same way as above

# Optimal transport

portions of code adapted from: https://github.com/cantinilab/OT-scOmics

In [ ]:
# using full mouse OE atlas with RPM tumors
adata = sc.read_h5ad('OE_atlas_with_ONB_tumors_all_cells.h5ad')

In [ ]:
#Filter out non-OE clusters (ie immune, fibroblasts, non OE lineage)
bad_clust=['Fibroblast', 'Lymphoid', 'Myeloid', 'Olfactory ensheathing', 'Pericyte']

#Filter out bad clusters
to_keep=(~adata.obs['cluster_names'].isin(bad_clust))

#Copy over to new anndata object
adata = adata[to_keep].copy()

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=3000,
    subset=False,
    layer="counts",
    flavor="seurat_v3",
    batch_key="mouse_ident"
)

In [ ]:
#Calculate highly variable genes using scVI methods
df_poisson = scvi.data.poisson_gene_selection(
    adata, n_top_genes=3000, batch_key="mouse_ident", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata.varm['df_poisson']= df_poisson

adata_query = adata[:, is_hvg].copy()
print(adata_query)

In [ ]:
# Downsample

#Define function
def obs_key_wise_subsampling(adata, obs_key, N):
    '''
    Subsample each class to same cell numbers (N). Classes are given by obs_key pointing to categorical in adata.obs.
    '''
    counts = adata.obs[obs_key].value_counts()
    # subsample indices per group defined by obs_key
    indices = [np.random.choice(adata.obs_names[adata.obs[obs_key]==group], size=N, replace=False) for group in counts.index]
    selection = np.hstack(np.array(indices))
    return adata[selection].copy()

In [ ]:
#Now run function with user defined N
adata_distance=obs_key_wise_subsampling(adata_query, 'cluster_names', 100)

In [ ]:
# Make sure we are working with raw counts
adata_distance.X=adata_distance.layers['counts']

In [ ]:
#Set clusters term within adata.obs
clusters=adata_distance.obs['cluster_names']
idx = np.argsort(clusters)

In [ ]:
#sc.pp.normalize_per_cell(adata_distance, counts_per_cell_after=1e4) #Note we do per cell normalization later
sc.pp.log1p(adata_distance)

In [ ]:
adata_df=adata_distance.to_df()

In [ ]:
#Now reconstruct adata object based on methods described in their paper
adata_df = adata_df.iloc[np.argsort(adata_df.std(1))[::-1][:1_100]] #last [:x] should be equal to # of cluster_names * 100 (11*100= 1_100)

In [ ]:
adata_model = ad.AnnData(adata_df)
adata_model.obs['cluster_names'] = clusters

In [ ]:
#Set clusters term within adata.obs
clusters=adata_model.obs['cluster_names']
idx = np.argsort(clusters)

In [ ]:
clusters_t=np.array(adata_model.obs)

In [ ]:
# Optionally, can specify categories order (this determines order in downstream matrix plot)
category_order_list=['HBC', 'Sustentacular', 'Bowmans Glands', 'MV ionocyte-like', 'MV tuft-like', 
                     'RPMA tumor','GBC', 'RPM tumor', 'INP', 'iOSN', 'mOSN']

In [ ]:
#Reorder cluster names for plotting
adata_model.obs['cluster_names'].cat.set_categories(category_order_list)

In [ ]:
#Set clusters term within adata.obs
clusters=adata_model.obs['cluster_names'].cat.set_categories(category_order_list)
idx = np.argsort(clusters)

In [ ]:
# Per-cell normalization (mandatory)
data_norm = adata_model.X.T.astype(np.double)
data_norm /= data_norm.sum(0)
# Add a small value to avoid numerical errors
data_norm += 1e-9
data_norm /= data_norm.sum(0)

In [ ]:
# set up cost matrix
C = otscomics.cost_matrix(adata_model.X.T.astype(np.double), 'cosine')

In [ ]:
# Compute OT distance matrix
# Need high memory and GPU for this, also if getting runtime error, just adjust batch_size
D_ot, errors = otscomics.OT_distance_matrix(
    data=data_norm, cost=C, eps=.1,
    dtype=torch.double, 
    device='cuda', batch_size=128
)

In [ ]:
# Check outputs
D_ot[idx][:,idx].shape

In [ ]:
cor_stack = D_ot[idx][:,idx]

In [ ]:
# plot matrix
#Hvgs set to 3000
plt.imshow(D_ot[idx][:,idx], vmax=0.04, #cmap='RdBu_r'
          )
plt.title('OT distance matrix')
plt.xlabel('Cells')
plt.ylabel('Cells')
plt.colorbar()
plt.yticks(np.arange(50, 1150, 100))
plt.xticks(np.arange(50, 1150, 100))
frame1=plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.yaxis.set_ticklabels([])

In [ ]:
# calculate average across each cluster
df_OT=pd.DataFrame(cor_stack)
df_OT_mean=df_OT.groupby(np.arange(len(df_OT))//100).mean()
df_OT_mean_T=df_OT_mean.T
df_OT_all_mean=df_OT_mean_T.groupby(np.arange(len(df_OT_mean_T))//100).mean()

In [ ]:
# rename index (this needs to match order that was input above)
# includes in vitro tumor
df_final=df_OT_all_mean.set_axis(['Sustentacular', 'HBC', 'MV tuft-like', 'MV ionocyte-like', 'Bowmans Glands','RPMA tumor',
                      'GBC','RPM tumor', 'INP', 'iOSN', 'mOSN'])

In [ ]:
# rename columns 
# this order needs to be the same as box immediately above
df_plot=df_final.rename(columns={0:'Sustentacular', 1:'HBC', 2:'MV tuft-like', 3:'MV ionocyte-like',
                                 4:'Bowmans Glands',5:'RPMA tumor', 6:'GBC', 7:'RPM tumor', 
                                 8:'INP', 9:'iOSN', 10:'mOSN'})

In [ ]:
# verify structure
df_plot

In [ ]:
# plot heatmap with average values

# set fontsize
plt.rcParams.update({'font.size': 10})

# set figure size
fig, ax = plt.subplots(figsize=(12,10))

# plot
sns.heatmap(data=df_plot, cmap='viridis',
            linewidths=0, linecolor='black',
           vmax=0.06, annot=True, #mask=mask
           )
sns.despine()
plt.xticks(rotation=90)

# can save fig, if desired
#fig.savefig('heatmap.png', dpi='figure', bbox_inches='tight')

In [ ]:
# plot clustermap
# this generates dendrogram to group cell types
sns.clustermap(data=df_plot, annot=True, cmap='viridis', vmax=0.05)

# GSEA

In [ ]:
# import additional packages required for running GSEA
import gseapy as gp
from gseapy import Biomart
bm = Biomart()

In [ ]:
# load in data

# here, for example, using RPM vs. RPMA gene lists derived from edgeR DE
gene_list = pd.read_csv('/hpc/group/goldsteinlab/Python/GSEA/rpm_rpma_de_edgeR_for_GSEA.csv')

In [ ]:
# select sample/condition gene list that you want
gene_list_RPM=gene_list[['RPM_gene']]

In [ ]:
# select top 200 genes 
gene_list_RPM=gene_list_RPM.drop(gene_list_RPM.index[200:])

In [ ]:
#Convert df or series to list
glist=gene_list_RPM.squeeze().str.strip().to_list()

In [ ]:
# run enrichr
enr_RPM = gp.enrichr(gene_list=glist,
                 gene_sets=['ChEA_2022', 
                            'ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X',
                            'GO_Biological_Process_2021',
                            'PanglaoDB_Augmented_2021',
                            'Tabula_Muris'],
                 organism='mouse',
                 outdir=None, 
                )

In [ ]:
# optionally can plot
dotplot(enr_RPM.results,
              column="Adjusted P-value",
              x='Gene_set', # set x axis, so you could do a multi-sample/library comparsion
              size=15,
              top_term=5,
              figsize=(3,7.5),
              title = "KEGG",
              xticklabels_rot=90, # rotate xtick labels
              show_ring=True, # set to False to revmove outer ring
              marker='o',
               save=True)

In [ ]:
# write results to csv output
enr_RPM.results.to_csv('RPM_Enrichr_table_edgeR_de.csv')